In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| code-fold: false
# IMPORTING LIBRARIES
# --------------------

# AstroML
from astroML.datasets import fetch_LINEAR_sample
from astropy.coordinates import SkyCoord
import astropy.units as u

# Basic libraries
import pickle
import os
import sys
from tqdm import tqdm

# Plotting
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties

# DataFrame analysis
import pandas as pd

# Math libraries
import numpy as np
import scipy as sc
from scipy.stats import norm

# CONFIGURATION
# -------------
sns.set_theme(style='white') # setting the theme for plotting
sys.path.insert(0,'../src/')
np.random.seed(42)

# configuring plotting colors
colors = ['#465BBB', '#A82F43', '#3F8FCE', '#7ABBCE', '#3A3865', '#612A37', '#DC5433', '#F29457']
b1 = '#465BBB'
b2 = '#3F8FCE'
b3 = '#7ABBCE'
b4 = '#3A3865'
black1 = '#22212A'
black2 = '#2D1E21'
or1 = '#A82F43'
or2 = '#612A37'
or3 = '#DC5433'
or4 = '#F29457'
muted_colors = sns.set_palette(sns.color_palette(colors))

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] = r'\usepackage{txfonts}'  # Use TX Times font

# Set up your font properties (optional)
font = FontProperties()
font.set_family('serif')
font.set_name('Times')  # Times font
%matplotlib inline 

In [3]:
#| code-fold: false
# Importing custom libraries
# ----------------------------
sys.path.insert(0,'../src/')
from BE_plotting import*
#from RR_lyrae_params import*
from blazhko_analysis import*

In [4]:
#| code-fold: false
# DATA
dataL = fetch_LINEAR_sample(data_home='../inputs') # fetching the data from astroML data library
end = 'ffinal'

# DATA from previous notebook
BE_STARS = pd.read_csv("../outputs/FINAL_BE_CATALOGUE.csv")
be_cand = pd.read_csv('../outputs/blazhko_canffinal.csv')
lc_analysis = pd.read_csv("../outputs/LC_ANALYSIS_"+end+".csv")
rrlyrae = pd.read_csv('../outputs/Lrrlyr_unprocessed.csv')
periodogr = pickle.load(open("../outputs/periodograms_"+end+".pkl", "rb"))
fits = np.load("../outputs/fits_"+end+".npy", allow_pickle=True)
ztf_data = np.load("../outputs/ztf_data_filter"+end+".npy", allow_pickle=True)

---

# Plotting the Blazhko candidates

In [5]:
blazhko_can = pd.read_csv('../outputs/lc_MASTER_bcan.csv')

In [6]:
blazhko_can.head()

,LINEAR ID,Plinear,Pztf,N_L,N_Z,L_chi2r,Z_chi2r,L_chi2,Z_chi2,Lampl,Zampl,BpeakL,BpeakZ,BperiodL,BperiodZ,Periodogram_f,B_score,Blazhko_f
0,29848,0.557020,0.557040,301,43,1.4,3.5,3.0,12.6,0.56,0.93,1.8328,1.7982,26.6205,333.3333,-,6,1
1,158779,0.609207,0.609189,293,616,1.6,3.9,3.7,34.2,0.47,0.68,1.6443,1.6444,352.7337,350.2627,-,6,1
2,263541,0.558218,0.558221,270,503,2.9,6.6,15.8,110.4,0.64,0.82,1.8621,1.8025,14.1513,89.9685,-,7,1
3,514883,0.557723,0.557737,289,555,1.7,5.5,5.3,53.7,0.55,0.72,1.8472,1.7958,18.4655,357.1429,-,7,1
4,523832,0.372376,0.372384,251,42,1.2,1.1,1.8,0.8,0.42,0.59,2.8040,2.7122,8.4370,37.3413,Z,0,2


In [7]:
blazhko_can.shape

(239, 18)

In [14]:
Lids = list(blazhko_can['LINEAR ID'])

In [15]:
len(Lids)

239

Note: u doljnjem crtanju koristim be_cand zato sto su mi sve funkcije u BE_plotting napravljene za specificna imena od pocetnih tablica koje mislim da nije potrebno mijenjati sada. Uostalom, lc_MASTER_bcan su iste zvijezde kao i u be_cand, samo s manje podataka jer je za prezentaciju.

In [ ]:
# CRTANJE SLIKA PO LISTI NEKIH ID-ova
# -------
for i in tqdm(range(len(Lids))):
    LID = Lids[i]
    for n, j in enumerate(fits):
            if j[0]==LID:
                break

    # select light curve fits
    L1 = fits[n][1][0]
    L2 = fits[n][1][2]

    # select the periodogram data
    for o, k in enumerate(periodogr):
            if k[0]==LID:
                break

    fL = periodogr[o][1][0]
    pL = periodogr[o][1][1]
    fZ = periodogr[o][2][0]
    pZ = periodogr[o][2][1]

    

    fFoldedL = periodogr[o][1][2]
    pFoldedL = periodogr[o][1][3]
    fFoldedZ = periodogr[o][2][2]
    pFoldedZ = periodogr[o][2][3]

    # selecting ZTF data
    lc = dataL.get_light_curve(LID)
    tL = lc.T[0]
    for f, g in enumerate(ztf_data):
        if g[0] == LID:
            break
    tZ = ztf_data[f][1]
    # ---------------------
    plotAll(LID, n, i, len(Lids), L1, L2, be_cand, fL, pL, fZ, pZ, fFoldedL, fFoldedZ, pFoldedL, pFoldedZ, dataL, tL, tZ, ztf_data, f='paper_support',plotSave=True)